# Hitters Veri Seti ile Makine Ogrenme Modelleri Kullanılarak Maaş Tahmini

1. Bağımlı Değişken:

Salary: 1986-1987 sezonunda kazanılan maaş
2. Açıklayıcı Değişkenler:

Hitters Veri setinde yer alan değişkenlerin açıklanması

AtBat: 1986-1987 sezonunda bir beyzbol sopası ile topa yapılan vuruş sayısı
Hits: 1986-1987 sezonundaki isabet sayısı
HmRun: 1986-1987 sezonundaki en değerli vuruş sayısı
Runs: 1986-1987 sezonunda takımına kaç sayı kazandırdı
RBI: Bir vurucunun vuruş yaptıgında kaç tane oyuncuya koşu yaptırdığı.
Walks: Karşı oyuncuya kaç defa hata yaptırdığı
Years: Oyuncunun major liginde kaç sene oynadığı
CAtBat: Oyuncunun kariyeri boyunca kaç kez topa vurduğu
CHits: Oyuncunun kariyeri boyunca kaç kez isabetli vuruş yaptığı
CHmRun: Oyucunun kariyeri boyunca kaç kez en değerli vuruşu yaptığı
CRuns: Oyuncunun kariyeri boyunca takımına kaç tane sayı kazandırdığı
CRBI: Oyuncunun kariyeri boyunca kaç tane oyuncuya koşu yaptırdığı
CWalks: Oyuncun kariyeri boyunca karşı oyuncuya kaç kez hata yaptırdığı
League: Oyuncunun sezon sonuna kadar oynadığı ligi gösteren A ve N seviyelerine sahip bir faktör
Division: 1986 sonunda oyuncunun oynadığı pozisyonu gösteren E ve W seviyelerine sahip bir faktör
PutOuts: Oyun icinde takım arkadaşınla yardımlaşma
Assits: 1986-1987 sezonunda oyuncunun yaptığı asist sayısı
Errors: 1986-1987 sezonundaki oyuncunun hata sayısı
Salary: Oyuncunun 1986-1987 sezonunda aldığı maaş(bin uzerinden)
NewLeague: 1987 sezonunun başında oyuncunun ligini gösteren A ve N seviyelerine sahip bir faktör
Gözlem Sayısı: 322

Değişken Sayısı: 20

In [ ]:
# Gerekli import işlemleri gerçekleştirildi. Doğrusal Regresyon modellerine ait import işlemleri yapıldı.
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from warnings import filterwarnings
filterwarnings('ignore')

# 1)Kesifci Veri Analizi

In [ ]:
df = pd.read_csv("../input/hitters/Hitters.csv")

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.nunique()


NewLeague,League ve Division degiskenleri kategorik degisken oldugu gozlendi (dtypes : object(3))   
Bu degiskenlerin kac kategori olduguna bakildiginda ise 2 farkli kategori oldugu gozlendi

In [ ]:
# Veri setinin korelasyonuna erişim sağlanmıştır. Değişkenler arasında ne tarz bir ilişki olduğu incelenmiştir. 
# Korelasyon değeri > 0 ise pozitif korelasyon bulunmaktadır. Bir değişken değeri artarken diğer değişkenin de değeri artmaktadır.
# Korelasyon = 0 ise korelasyon yoktur anlamına gelmektedir.
# Korelasyon < 0 ise negatif korelasyon bulunmaktadır. Bir değişken artarken diğer değişken azalmaktadır. 
# Korelasyonlar incelendiğinde Salary bağımlı değişkene pozitif korelasyon olarak etkimekte olan 2 değişken bulunmaktadır. 
# Bu değişkenler CRBI ile CRuns değişkenleridir. Bunlar arttıkça Salary (Maaş) değişkeni de artmaktadır.
df.corr()


In [ ]:
#League NewLeague ve Division değişkenlerine göre groupby yapılıp ortalama ne kadar maaş alınmakta oldugu incelenmiştir.
df.groupby(["League", "Division","NewLeague"]).agg({"Salary": "mean"})

In [ ]:
df["Salary"].mean()

In [ ]:
df["Salary"].median()

In [ ]:
df["Salary"].std()

In [ ]:
df["Salary"].max()

In [ ]:
df["Salary"].min()

In [ ]:
# Bağımlı değişkenin betimsel istatistiklerine erişim sağlanmıştır.
df["Salary"].describe([0.05,0.1,0.25,0.75,0.9,0.95])

Maas degiskenin ortalamasinin standart sapma verisine yakin bir deger olmasi verinin  dengesiz dagildigini
gosterebilir. Yine de ortalama ve medyan degerinin birbirine yakin olmasi hesaba katildiginda verinin cok da dengesiz
dagilmadigi Fakat belki istisnai birkac outlyer degerler(maximum ve minumum deger gibi) std sapmayi artirmis olabilecegi yorumunu 
yapabiliriz.

In [ ]:
#Salary (Maaş) değişkeninin histogram ve yoğunluk grafiği olusturulmustur.
sns.distplot(df.Salary);

In [ ]:
# Veri setinin korelasyon matrisi grafiğine erişim sağlanmıştır.
f, ax = plt.subplots(figsize= [20,15])
sns.heatmap(df.corr(), annot=True, fmt=".2f", ax=ax, cmap = "magma" )
ax.set_title("Correlation Matrix", fontsize=20)
plt.show()

In [ ]:
df_corr =df.corr()
df_corr["Salary"].sort_values( ascending = True)

#Her ne kadar bu alanda domain bilgisine sahip olmasam da korelasyon matrisini yorumlayacak olursam 
#assist ve Errors degiskenleri ile Salary arasindaki korelasyonlari cok kucuk bir deger oldugundan 
#assist ve Errors maasi en az etkilyen faktorlerdir diyebilir miyiz?

Her ne kadar bu alanda domain bilgisine sahip olmasam da korelasyon matrisini yorumlayacak olursam 
assist ve Errors degiskenleri ile Salary arasindaki korelasyonlari cok kucuk bir deger oldugundan 
assist ve Errors maasi en az etkilyen faktorlerdir diyebilir miyiz?

In [ ]:
df_corr["AtBat"].sort_values( ascending = False)
#AtBat ve Hits degiskenleri korelasyonlari cok yuksek oldugundan birini ucurmak dogru olur mu?

# 2) Veri Onisleme

**a)  Kategorik Veri Onislemesi ---- One Hot Encoding Dönüşümü******

In [ ]:
# Veri setinde yer alan kategorik değişkenlerin sayısal değerlere dönüştürülmesi gerekmektedir. 
# Bu nedenle bu dönüşüm işlemleri Label Encoding ve One Hot Encoding yöntemi ile yapılmaktadır.
# Burada One Hot Encoding dönüşümü yapılarak kategorik değişkenler sayısal değerlere dönüşülmüştür. 
# Aynı zamanda Dummy değişken tuzağından da drop_first = True kullanarak korunulmuştur.
df = pd.get_dummies(df, columns =["Division","League","NewLeague"], drop_first = True)

In [ ]:
df.head()

b) Eksik Gözlem Analizi 

In [ ]:
df.isnull().sum()

In [ ]:
# Eksik gözlemler KNN Algoritması ile dolduruluyor.
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 3)
df_filled = imputer.fit_transform(df)

In [ ]:
# Tekrar sorgulandı eksik değer var mı diye hepsinin KNN algoritması ile doldurulduğu gözlemlendi.
df = pd.DataFrame(df_filled,columns = df.columns)
df.isnull().sum()

c) Aykiri Gozlem Analizi

In [ ]:
# Salary değişkenini boxplot yöntemi ile görselleştirme işlemi yapılmıştır. Aykırı gözlemlere grafikte gozlemliyoruz.
import seaborn as sns
sns.boxplot(x = df["Salary"]);

In [ ]:
# 10 lik çeyreklik ve % 90 lik çeyreklikler incelendiğinde aykırı gözleme rastlanmıştır.

# Aykiri gozlem rastlana  değişkeni için akırı gözlem analizinde kullanılan IQR derinin hesaplanması 
# ve alt ve üst sınırların belirlenilmesi işlemi gerçekleştirilmiştir.
# Bulunan aykırı gözlemler üst sınır ile baskılanmıştır. 
# Aykırı gözlem analizinde en yaygın olarak baskılama yöntemi kullanılmaktadır. 

for feature in df:

    Q1 = df[feature].quantile(0.1)
    Q3 = df[feature].quantile(0.9)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
        print("Aykiri gozlem baskilandiktan sonra kontrol")
    else:
        print(feature, "no")
    df.loc[df[feature] < lower,feature] = lower
    df.loc[df[feature] > upper,feature] = upper
    
    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])    
    else:
        print(feature, "no")

c.1) Local Outlier Factor Yöntemi ile Aykırı Gözlem Analizi (LOF)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
lof =LocalOutlierFactor(n_neighbors= 20)
lof.fit_predict(df.drop(columns="Salary"))

In [ ]:
# Skor değerleri gelmiştir.
df_scores = lof.negative_outlier_factor_
np.sort(df_scores)[0:10]

In [ ]:
threshold = np.sort(df_scores)[1]
threshold

In [ ]:
# Bir tane lof degeri bulundu
df[df_scores < threshold]

In [ ]:
# Belirlenen eşik değer veri setine uyarlanarak aykırı gözlemlerden kurtulunmuş olundu.outlier = df_scores > threshold
outlier = df_scores > threshold
df = df[outlier]

In [ ]:
df.shape

# 3) Modeleme

In [ ]:

y = df["Salary"]
X = df.drop('Salary', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning)
#!pip install lightgbm
#!pip install xgboost
#!pip install catboost
#conda install -c conda-forge lightgbm

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale 
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
import xgboost
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from lightgbm import LGBMRegressor

In [ ]:
models = []
models.append(('LR', LinearRegression()))
models.append(('RR', Ridge()))
models.append(('LR', Lasso()))
models.append(('ER', ElasticNet()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('CART', DecisionTreeRegressor()))
models.append(('RF', RandomForestRegressor()))
models.append(('SVR', SVR()))
models.append(('GBM', GradientBoostingRegressor()))
models.append(("XGBoost", XGBRegressor()))
models.append(("CatBoost", CatBoostRegressor(verbose = False)))
models.append(("LightGBM", LGBMRegressor()))

In [ ]:
for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        msg = "%s: (%f)" % (name, rmse)
        print(msg)

In [ ]:
for name, model in models:
        model.fit(X, y)
        y_pred = model.predict(X)
        rmse = np.sqrt(mean_squared_error(y, y_pred))
        msg = "%s: (%f)" % (name, rmse)
        print(msg)

In [ ]:
for name, model in models:
        rmse = np.sqrt(np.mean(-cross_val_score(model, X, y, cv = 10, scoring = "neg_mean_squared_error")))
        msg = "%s: (%f)" % (name, rmse)
        print(msg)

# 3) Model Tuning

a) Hiperparametre optimizasyonu

In [ ]:
lgb_model = LGBMRegressor()
lgb_model

In [ ]:
lgbm_params = {"learning_rate": [0.001,0.002, 0.003, 0.0005, 0.0001],
              "n_estimators": [200,500,1000,5000],
              "max_depth": [2,4,6,7,10],
              "colsample_bytree": [1,0.8,0.5,0.4]}

In [ ]:
lgbm_cv_model = GridSearchCV(lgb_model, 
                             lgbm_params, 
                             cv = 2, 
                             n_jobs = -1, 
                             verbose =2).fit(X_train, y_train)

In [ ]:
lgbm_cv_model.best_params_

In [ ]:
lgbm_tuned = LGBMRegressor(**lgbm_cv_model.best_params_).fit(X_train, y_train)

In [ ]:
np.sqrt(mean_squared_error(y_test, lgbm_tuned.predict(X_test)))

In [ ]:
Importance = pd.DataFrame({'Importance':lgbm_tuned.feature_importances_*100}, 
                          index = X_train.columns)


Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', )

plt.xlabel('Variable Importance')
plt.gca().legend_ = None

b) Feeature Engineering

Yeni değişkenler oluşturmak modeller için önem arz etmektedir. Ancak mantıklı bir yeni değişken oluşturmak gerekir. 
Bu veri setinde oyuncunun kariyerine bağlı değişkenler yer almaktadır ve kariyer süresi yer almaktadır. 
Bu durumda kariyere bağlı değişkenler / kariyer süresi yaparak  elde edilen  değerler mantıklı bir değişken oluşturulabilir.
Aşağıda yapılan işlemde bunu ele alıyor olacağız.

In [ ]:
df["NEW_catbat"] = df["CAtBat"]/df["Years"]
df["NEW_chits"] =df["CHits"]/df["Years"]
df["NEW_chmrun"] = df["CHmRun"]/df["Years"]
df["NEW_cruns"] = df["CRuns"]/df["Years"]
df["NEW_crbi"] = df["CRBI"]/df["Years"]
df["NEW_cwalks"]=df["CWalks"]/df["Years"]
y = df["Salary"]
X = df.drop('Salary', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state=46)

In [ ]:
for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        msg = "%s: (%f)" % (name, rmse)
        print(msg)

In [ ]:
df.head()

Scale

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(data=scaler.transform(df),index = df.index,columns=df.columns)

In [ ]:
df.head()

In [ ]:
for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        msg = "%s: (%f)" % (name, rmse)
        print(msg)